In [7]:
import pandas as pd

In [8]:
pokemon_lendary = ['Enamorus','Raikou','Calyrex','Kyurem','Regirock','Zeraora','Tapu Koko','Suicune','Terrakion','Nihilego','Diancie','Azelf','Arceus','Mew','Zapdos','Xurkitree','Jirachi','Mewtwo','Tornadus','Keldeo','Thundurus','Landorus','Cobalion','Mesprit','Galarian Zapdos','Galarian Moltres','Zamazenta','Kartana','Magearna','Stakataka','Tapu Lele','Silvally','Cosmoem','Necrozma','Rayquaza','Regice','Volcanion','Zarude','Dialga','Regidrago','Solgaleo','Regieleki','Eternatus','Tapu Fini','Palkia','Lunala','Celebi','Pheromosa','Zacian','Xerneas','Hoopa','Reshiram','Genesect','Yveltal','Deoxys','Cresselia','Glastrier','Naganadel','Marshadow','Melmetal','Latios','Urshifu-rapid-strike','Buzzwole','Type: Null','Meloetta','Giratina','Virizion','Phione','Regigigas','Kyogre','Latias','Lugia','Entei','Blacephalon','Galarian Articuno','Victini','Spectrier','Shaymin','Ho-Oh','Celesteela','Uxie','Cosmog','Guzzlord','Articuno','Darkrai','Zekrom','Registeel','Groudon','Moltres','Tapu Bulu','Manaphy','Zygarde','Heatran','Miraidon','Koraidon','Chi-yu','Wo-chien','Ting-lu','Chien-pao','Iron Leaves','Walking Wake']

df_pokemon_all = pd.read_excel('pokemon_data.xlsx')

In [9]:
classificados = []

df_pokemon_all['Is_legendary'] = False

for i in pokemon_lendary:
    if i in df_pokemon_all['Name'].values:
        print(f'Lista: {i} \n Dataframe: {df_pokemon_all["Name"].values}')
        df_pokemon_all.loc[df_pokemon_all['Name'] == i, 'Is_legendary'] = True

df_pokemon_all.loc[df_pokemon_all['Is_legendary'] == True]

Lista: Raikou 
 Dataframe: ['Bulbasaur' 'Ivysaur' 'Venusaur' ... 'Miraidon' 'Walking Wake'
 'Iron Leaves']
Lista: Calyrex 
 Dataframe: ['Bulbasaur' 'Ivysaur' 'Venusaur' ... 'Miraidon' 'Walking Wake'
 'Iron Leaves']
Lista: Kyurem 
 Dataframe: ['Bulbasaur' 'Ivysaur' 'Venusaur' ... 'Miraidon' 'Walking Wake'
 'Iron Leaves']
Lista: Regirock 
 Dataframe: ['Bulbasaur' 'Ivysaur' 'Venusaur' ... 'Miraidon' 'Walking Wake'
 'Iron Leaves']
Lista: Zeraora 
 Dataframe: ['Bulbasaur' 'Ivysaur' 'Venusaur' ... 'Miraidon' 'Walking Wake'
 'Iron Leaves']
Lista: Tapu Koko 
 Dataframe: ['Bulbasaur' 'Ivysaur' 'Venusaur' ... 'Miraidon' 'Walking Wake'
 'Iron Leaves']
Lista: Suicune 
 Dataframe: ['Bulbasaur' 'Ivysaur' 'Venusaur' ... 'Miraidon' 'Walking Wake'
 'Iron Leaves']
Lista: Terrakion 
 Dataframe: ['Bulbasaur' 'Ivysaur' 'Venusaur' ... 'Miraidon' 'Walking Wake'
 'Iron Leaves']
Lista: Nihilego 
 Dataframe: ['Bulbasaur' 'Ivysaur' 'Venusaur' ... 'Miraidon' 'Walking Wake'
 'Iron Leaves']
Lista: Diancie 
 Datafra

,Pokedex ID,Name,Type_1,Type_2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Is_legendary
191,144,Articuno,Ice,Flying,580,90,85,100,95,125,85,True
193,145,Zapdos,Electric,Flying,580,90,90,85,125,90,100,True
195,146,Moltres,Fire,Flying,580,90,100,90,125,85,90,True
200,150,Mewtwo,Psychic,Psychic,680,106,110,90,154,90,130,True
203,151,Mew,Psychic,Psychic,600,100,100,100,100,100,100,True
...,...,...,...,...,...,...,...,...,...,...,...,...
1067,898,Calyrex,Psychic,Grass,500,100,80,80,80,80,80,True
1190,1007,Koraidon,Fighting,Dragon,670,100,135,115,85,100,135,True
1191,1008,Miraidon,Electric,Dragon,670,100,85,100,135,115,135,True
1192,1009,Walking Wake,Water,Dragon,590,99,83,91,125,83,109,True
